# Local Web Server Setup
First, let's create a simple web server to serve your content. You can either:
1. Put your content in a file and serve it, or
2. Create a simple server with some test content

We'll create both options below. Choose the one that fits your needs.

In [ ]:
# Option 1: Create a simple HTML file with your content
import os

# Create a sample HTML file
html_content = """
<!DOCTYPE html>
<html>
<body>
    <main>
        <h1>Sample Content</h1>
        <p>This is some sample content that will be sent to ChatGPT.</p>
        <p>Replace this content with your own text or paste your content here.</p>
    </main>
</body>
</html>
"""

# Save the HTML file
with open('content.html', 'w') as f:
    f.write(html_content)

print("Created content.html - you can edit this file to add your own content")

Created content.html - you can edit this file to add your own content


In [10]:
# Option 2: Start a simple HTTP server
import http.server
import socketserver
import threading
import time  # Added time module import

def start_server(port=8000):
    """Start a simple HTTP server in a separate thread"""
    Handler = http.server.SimpleHTTPRequestHandler
    
    try:
        with socketserver.TCPServer(("", port), Handler) as httpd:
            print(f"\nServer started at http://localhost:{port}")
            print("You can now run the main script to fetch content")
            print("Press Ctrl+C in this cell's output to stop the server")
            httpd.serve_forever()
    except OSError as e:
        if "Address already in use" in str(e):
            print(f"\nError: Port {port} is already in use.")
            print("This might mean:")
            print("1. The server is already running (which is good!)")
            print("2. Another program is using this port")
            print("\nTry using the main script now to fetch content.")
        else:
            raise e

# Start the server in a background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print("Local server starting...")
time.sleep(1)  # Give the server a moment to start

Local server starting...

Error: Port 8000 is already in use.
This might mean:
1. The server is already running (which is good!)
2. Another program is using this port

Try using the main script now to fetch content.


In [ ]:
# Chrome Profile Settings
CHROME_PROFILE_DIR = "/Users/ramihuunguyen/Library/Application Support/Google/Chrome"
CHROME_PROFILE = "Default"
WINDOW_SIZE = "1400,1000"
WAIT_TIME = 2

import os
import time
import subprocess
import sys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Try to import undetected_chromedriver
try:
    import undetected_chromedriver as uc
except ImportError:
    print("undetected_chromedriver not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "undetected-chromedriver"])
    import undetected_chromedriver as uc

def check_chrome_processes():
    """Check if Chrome processes are running"""
    try:
        # Check for Chrome processes on macOS
        result = subprocess.run(['pgrep', '-f', 'Google Chrome'], 
                              capture_output=True, text=True)
        return len(result.stdout.strip()) > 0
    except:
        return False

def kill_chrome_processes():
    """Kill all Chrome processes"""
    try:
        subprocess.run(['pkill', '-f', 'Google Chrome'], check=False)
        time.sleep(2)
        print("Chrome processes terminated.")
    except:
        print("Could not automatically close Chrome. Please close it manually.")

def setup_chrome():
    """Setup Chrome with user's existing profile"""
    options = uc.ChromeOptions()
    
    # Add profile arguments
    options.add_argument(f'--user-data-dir={CHROME_PROFILE_DIR}')
    options.add_argument(f'--profile-directory={CHROME_PROFILE}')
    options.add_argument(f'--window-size={WINDOW_SIZE}')
    
    # Additional stability arguments
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-extensions-except')
    options.add_argument('--disable-default-apps')
    
    try:
        # Create driver with version_main parameter for better compatibility
        driver = uc.Chrome(options=options, headless=False, version_main=None)
        return driver
    except Exception as e:
        print(f"Error creating Chrome driver: {e}")
        raise

def open_chatgpt():
    driver = None
    try:
        # Check if Chrome is running
        if check_chrome_processes():
            print("\nChrome is currently running.")
            choice = input("Do you want to automatically close it? (y/n): ").lower()
            if choice == 'y':
                kill_chrome_processes()
            else:
                print("Please close Chrome manually and try again.")
                return
        
        # Verify profile directory exists
        if not os.path.exists(CHROME_PROFILE_DIR):
            print(f"\nError: Chrome profile directory not found: {CHROME_PROFILE_DIR}")
            print("Please check your Chrome profile path.")
            return
        
        # Start Chrome
        print("\nStarting Chrome with your profile...")
        driver = setup_chrome()
        
        # Wait a moment for Chrome to fully load
        time.sleep(WAIT_TIME)
        
        # Navigate to ChatGPT
        print("Navigating to ChatGPT...")
        driver.get("https://chatgpt.com")
        
        # Wait for page to load
        time.sleep(WAIT_TIME)
        
        print("\nSuccess! ChatGPT should be loaded.")
        print("The browser will stay open for you to use.")
        print("Press Enter in this terminal when you want to close the browser...")
        
        input()
        
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("\nTroubleshooting tips:")
        print("1. Make sure Chrome is completely closed before running this script")
        print("2. Check if your Chrome profile path is correct")
        print("3. Try running: pip install --upgrade undetected-chromedriver")
        print("4. Make sure you have the latest version of Chrome installed")
        
    finally:
        if driver:
            try:
                driver.quit()
                print("Browser closed successfully.")
            except:
                print("Browser may have already been closed.")

def main():
    print("Chrome ChatGPT Opener")
    print("=" * 30)
    
    try:
        open_chatgpt()
    except KeyboardInterrupt:
        print("\nOperation cancelled by user.")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()